In [ ]:
import numpy as np
from numba import njit
import matplotlib.pyplot as plt

# ─────────────────────────────────────
# 1. Global constants & Pauli operators
# ─────────────────────────────────────
PI        = np.pi
B0     = 1.0
omega0 = 1.0
h_FD      = 1.0e-4
total_T   = 2.0
N_slices  = 500
tau       = total_T / N_slices
theta_rot = PI / 3
n_epochs  = 100
J         = 0.01

# Pauli matrices
σx = np.array([[0,1],[1,0]],complex)
σy = np.array([[0,-1j],[1j,0]],complex)
σz = np.array([[1,0],[0,-1]],complex)
I2 = np.eye(2, dtype=complex)

σ1x,σ1y,σ1z = np.kron(σx,I2),np.kron(σy,I2),np.kron(σz,I2)
ZZ          = np.kron(σz,σz)

# Bell-state projectors
bell = np.array([[1,0,0,-1],
                 [0,1,1,0],
                 [0,1,-1,0],
                 [1,0,0,1]],complex)/np.sqrt(2)
projs = np.array([np.outer(v,v.conj()) for v in bell])

ψ0  = np.array([1,0,0,1],complex)/np.sqrt(2)
ρ0  = np.outer(ψ0,ψ0.conj())

# ─────────────────────────────────────
# 2. Unitary block -— target = −Bcos σx − Bsin σz + JZZ
# ─────────────────────────────────────
@njit(inline='always')
def U4_full(vx, vy, vz, Jcouple, dt):
    hz = np.array([vz+Jcouple, vz-Jcouple])
    U  = np.zeros((4,4),dtype=np.complex128)
    for blk in range(2):
        mag = np.sqrt(vx*vx + vy*vy + hz[blk]*hz[blk])
        if mag < 1e-14:
            cs=1.0; ss=0.0
        else:
            cs = np.cos(mag*dt)
            ss = np.sin(mag*dt)/mag
        a = cs-1j*ss*hz[blk]; d = cs+1j*ss*hz[blk]
        b = -1j*ss*(vx-1j*vy); c = -1j*ss*(vx+1j*vy)
        if blk==0:
            U[0,0]=a; U[0,2]=b; U[2,0]=c; U[2,2]=d
        else:
            U[1,1]=a; U[1,3]=b; U[3,1]=c; U[3,3]=d
    return U

@njit
def probabilities(theta_rot, tau, n_slices,
                  B, ω,
                  Vx, Vy, Vz, Jcouple,
                  projs):
    ρ = ρ0.copy()
    for t in range(n_slices):
        t_now = t*tau
        # --- target fields ---
        vx_t = -B * np.cos(ω*t_now)
        vy_t = 0.0
        vz_t = -B * np.sin(ω*t_now)
        # --- control fields ---
        vx_t += Vx[t]; vy_t += Vy[t]; vz_t += Vz[t]
        U  = U4_full(vx_t, vy_t, vz_t, Jcouple, tau)
        ρ  = U @ ρ @ U.conj().T

    # measurement-axis rotation
    c = 1/np.sqrt(3)
    Urot = U4_full(c,c,c,0.0,theta_rot)
    ρ = Urot @ ρ @ Urot.conj().T

    p = np.empty(4)
    for k in range(4):
        p[k] = np.real(np.trace(ρ @ projs[k]))
    return p

@njit
def f0_and_trace(Vx,Vy,Vz,
                 theta_rot,tau,n_slice,
                 B0,ω0,h,Jcouple,projs):
    # base + ±h on B and ω
    P0   = probabilities(theta_rot,tau,n_slice,B0,   ω0,   Vx,Vy,Vz,Jcouple,projs)
    PBp  = probabilities(theta_rot,tau,n_slice,B0+h, ω0,   Vx,Vy,Vz,Jcouple,projs)
    PBm  = probabilities(theta_rot,tau,n_slice,B0-h, ω0,   Vx,Vy,Vz,Jcouple,projs)
    Pwp  = probabilities(theta_rot,tau,n_slice,B0,   ω0+h,Vx,Vy,Vz,Jcouple,projs)
    Pwm  = probabilities(theta_rot,tau,n_slice,B0,   ω0-h,Vx,Vy,Vz,Jcouple,projs)

    grad = np.empty((2,4))
    grad[0]=(PBp-PBm)/(2*h)
    grad[1]=(Pwp-Pwm)/(2*h)

    F=np.zeros((2,2))
    for i in range(2):
        for j in range(2):
            acc=0.0
            for k in range(4):
                acc += grad[i,k]*grad[j,k]/max(P0[k],1e-12)
            F[i,j]=acc
    Cov = np.linalg.inv(F)
    tr_inv = Cov[0,0]+Cov[1,1]
    f0 = 1.0/(1/F[0,0]+1/F[1,1])
    return f0,tr_inv

# ─────────────────────────────────────
# 3. Initial control: Optimal control for H0 without the Ising term
# ─────────────────────────────────────
B_ctrl   = B0
ω_ctrl   = omega0

V1x=np.zeros(N_slices); V1y=np.zeros(N_slices); V1z=np.zeros(N_slices)
for t in range(N_slices):
    t_now=t*tau
    V1x[t] =  B_ctrl*np.cos(ω_ctrl*t_now)
    V1y[t] = -ω_ctrl/2
    V1z[t] =  B_ctrl*np.sin(ω_ctrl*t_now)

# ─────────────────────────────────────
# 3. Initial control: Control obtained by GRAPE in the former case
# ─────────────────────────────────────
# V1x=np.zeros(N_slices); V1y=np.zeros(N_slices); V1z=np.zeros(N_slices)
# V1x = [-3.7250336236, -0.8543293145, 2.3105404949, 5.5184443579, 8.3027328975, 10.1656894237, 10.8210207626, 10.3412090927, 9.0937961915, 7.5346068725, 6.0507860833, 4.9160664571, 4.2876839677, 4.1932923610, 4.5539867905, 5.2329324773, 6.0416278786, 6.7166424907, 6.9535236139, 6.5078256540, 5.3024455983, 3.4778462210, 1.3564483592, -0.6547037844, -2.1854961999, -3.0123447613, -3.1233737500, -2.7018222577, -2.0261615038, -1.3473527612, -0.8152686041, -0.4758035713, -0.3071006129, -0.2576954649, -0.2724573231, -0.3060606836, -0.3277310436, -0.3220092885, -0.2833596515, -0.2136763673, -0.1180936265, -0.0037584911, 0.1211909126, 0.2479165473, 0.3683757297, 0.4750221023, 0.5625025590, 0.6277263906, 0.6698560608, 0.6904305142, 0.6929858581, 0.6821775948, 0.6626599407, 0.6391470102, 0.6155024725, 0.5950506471, 0.5796452196, 0.5711747018, 0.5702159656, 0.5774565149, 0.5925393383, 0.6143684784, 0.6415995022, 0.6717619450, 0.7013525326, 0.7269184307, 0.7444918817, 0.7499694304, 0.7401331174, 0.7122696931, 0.6649532557, 0.5982128347, 0.5133853545, 0.4129357599, 0.3010383113, 0.1821875506, 0.0616996908, -0.0550207968, -0.1626098002, -0.2562865474, -0.3318734382, -0.3863182722, -0.4176381339, -0.4246752902, -0.4074257466, -0.3665013160, -0.3037832333, -0.2219871546, -0.1256875483, -0.0210685419, 0.0830770024, 0.1763870707, 0.2470895172, 0.2838558109, 0.2775464638, 0.2236437574, 0.1237484908, -0.0132615679, -0.1713710093, -0.3298511084, -0.4659861796, -0.5607282403, -0.6041188928, -0.6007816705, -0.5727458309, -0.5562803086, -0.5923409079, -0.7130477818, -0.9260797883, -1.2078978747, -1.5065860235, -1.7554087585, -1.8915685874, -1.8722315777, -1.6836281443, -1.3415812670, -0.8875257294, -0.3804959166, 0.1120681277, 0.5244139343, 0.8065118004, 0.9365539823, 0.9291003936, 0.8323271976, 0.7114350018, 0.6250879336, 0.6064128839, 0.6636168908, 0.7936012389, 1.0009202327, 1.3013123598, 1.7107181440, 2.2288101135, 2.8348001056, 3.4925147141, 4.1621515940, 4.8056416635, 5.3906628703, 5.8941012034, 6.3087022817, 6.6446087265, 6.9248621281, 7.1738007035, 7.4061876822, 7.6227811894, 7.8144234062, 7.9703522553, 8.0871650816, 8.1708861108, 8.2302936158, 8.2683729430, 8.2761687657, 8.2361364097, 8.1328453351, 7.9647781723, 7.7526351021, 7.5388995510, 7.3807922733, 7.3389559544, 7.4624942549, 7.7711848023, 8.2362819563, 8.7710593422, 9.2417734358, 9.5064163653, 9.4669102098, 9.1072349119, 8.4976294181, 7.7644671134, 7.0456420385, 6.4524685565, 6.0508937011, 5.8669684837, 5.9062574421, 6.1634491111, 6.5868365137, 7.0025549304, 7.0779024137, 6.4236690361, 4.8122531885, 2.3254245291, -0.7106590472, -3.8798622428, -6.7462797278, -8.8807595694, -9.9876816278, -10.1008401161, -9.6121516307, -9.0144528489, -8.5899792713, -8.3374179820, -8.1266679918, -7.8576897139, -7.5033887412, -7.0784897560, -6.6086628337, -6.1213858098, -5.6452706444, -5.2017728439, -4.7929196219, -4.4011147183, -4.0026579317, -3.5855592941, -3.1584788997, -2.7448805736, -2.3747691054, -2.0736452265, -1.8550661549, -1.7172333436, -1.6436542339, -1.6084089859, -1.5827379718, -1.5425363300, -1.4727278661, -1.3687347634, -1.2338544311, -1.0766129845, -0.9071456034, -0.7365891422, -0.5765072715, -0.4378014116, -0.3287157401, -0.2530341156, -0.2094696847, -0.1915285466, -0.1888131042, -0.1900343344, -0.1837580295, -0.1613606753, -0.1177382565, -0.0515345688, 0.0350305076, 0.1369263680, 0.2462410198, 0.3540850173, 0.4507930031, 0.5277561768, 0.5793013102, 0.6033709393, 0.6016264230, 0.5785488712, 0.5402543977, 0.4916899881, 0.4352582797, 0.3691315557, 0.2869800898, 0.1797510204, 0.0419294966, -0.1225735941, -0.2947019670, -0.4412818117, -0.5234275620, -0.5087189062, -0.3815153304, -0.1486255939, 0.1620140465, 0.5059693639, 0.8314440027, 1.0888412052, 1.2442618821, 1.2928033011, 1.2683176127, 1.2385768750, 1.2816265525, 1.4481745094, 1.7288913018, 2.0487913694, 2.2967635420, 2.3701159700, 2.2118558934, 1.8256173309, 1.2685047141, 0.6334936625, 0.0262669352, -0.4572705943, -0.7541985259, -0.8549187064, -0.8092986050, -0.7098776921, -0.6474500594, -0.6597534066, -0.7093869298, -0.7104732243, -0.5842258060, -0.3043896404, 0.0887684287, 0.5058446186, 0.8411373739, 1.0102117471, 0.9763808593, 0.7566815319, 0.4073210452, 0.0005910036, -0.3919868204, -0.7060584764, -0.8835508906, -0.8795388741, -0.6897777527, -0.3813965510, -0.0942452765, 0.0005438426, -0.2397516151, -0.8785073183, -1.8771456682, -3.1022359362, -4.3539406495, -5.4082610811, -6.0677181519, -6.2173049392, -5.8773329817, -5.2248186082, -4.5529911879, -4.1694472433, -4.2838986098, -4.9417831781, -6.0189748536, -7.2530948415, -8.2855405631, -8.7196234185, -8.2221588853, -6.6604928887, -4.1955874173, -1.2518437850, 1.6179425014, 3.8883062641, 5.1873073765, 5.3793731245, 4.5962301374, 3.2141317961, 1.7639032040, 0.7340231757, 0.3650809127, 0.6392933690, 1.4066602306, 2.4322996050, 3.4062573353, 4.0133427674, 4.0264767654, 3.3744925146, 2.1651275713, 0.6599497848, -0.7958044535, -1.8778974476, -2.3968170571, -2.3723301798, -2.0014783751, -1.5152567586, -1.0414167704, -0.5913465665, -0.1425818135, 0.2942183788, 0.6673539437, 0.9108880120, 0.9768956034, 0.8597812951, 0.6029573946, 0.2848124730, -0.0096214686, -0.2175454386, -0.3177936018, -0.3325629733, -0.3109862113, -0.2994168089, -0.3144759214, -0.3360413172, -0.3218297661, -0.2322152681, -0.0475542771, 0.2254017394, 0.5559591080, 0.8981330732, 1.2002145870, 1.4145881700, 1.5098273198, 1.4800706992, 1.3486263582, 1.1607679420, 0.9673015443, 0.8062023698, 0.6895419194, 0.6036656321, 0.5202633194, 0.4140498586, 0.2744435937, 0.1083440096, -0.0656102243, -0.2244881222, -0.3471641863, -0.4189509643, -0.4338584223, -0.3946728970, -0.3105062432, -0.1927757295, -0.0532924402, 0.0945777839, 0.2333367250, 0.3423117101, 0.4027555547, 0.4045296283, 0.3494354209, 0.2497988412, 0.1233762647, -0.0112246007, -0.1388892800, -0.2507688143, -0.3461814714, -0.4332469481, -0.5273470743, -0.6462540234, -0.8034868997, -0.9996627963, -1.2153879574, -1.4085761353, -1.5234973505, -1.5105388200, -1.3457529652, -1.0402374481, -0.6364564935, -0.1979605937, 0.2028240762, 0.4998881879, 0.6532967798, 0.6690834124, 0.6066661637, 0.5593170172, 0.6119178812, 0.7937389023, 1.0531558976, 1.2713440266, 1.3153474502, 1.0998269486, 0.6189364744, -0.0606485361, -0.8283426194, -1.5601839785, -2.1445738768, -2.5067582074, -2.6339329075, -2.5920292492, -2.5094282199, -2.5172912557, -2.6819026455, -2.9780457938, -3.3066355539, -3.5261368045, -3.4975043952, -3.1389093034, -2.4608273792, -1.5618753819, -0.5980728920, 0.2546865796, 0.8360954815, 1.0416096052, 0.8766071045, 0.5060959906, 0.2157022637, 0.2204602848, 0.4835796824, 0.8098627874, 1.0692233785, 1.2155861992, 1.1989420536, 0.9764340328, 0.5584064695, 0.0037903013, -0.6102019758, -1.2048786142, -1.7097373394, -2.0896998860, -2.3972710200, -2.7923226454, -3.4210923607, -4.1819078250, -4.6584011101, -4.4057018635, -3.3221243141, -1.7274316026, -0.1574758528, 0.8664855075, 0.9867369250, 0.1158249441, -1.5034740319, -3.3633306808, -4.9088351567, -5.7927403566, -5.9775986574, -5.6282694840, -4.9564682589, -4.1673473252, -3.4590502436, -2.9984749354, -2.8904661545, -3.1644730475, -3.7686953107, -4.5707215247, -5.3866266408, -6.0619343075, -6.6138983403, -7.4546717690, -9.6345068317]
# V1y = [-7.0831823774, -5.5631822351, -4.1697482012, -3.0452498998, -2.2791879234, -1.8703108324, -1.7007057271, -1.5911207407, -1.4152362630, -1.1829877974, -1.0090664480, -0.9865476360, -1.0865558546, -1.1864448089, -1.1862356114, -1.0736336193, -0.8955921614, -0.7159300647, -0.5919927363, -0.5431717408, -0.5315785794, -0.4798244040, -0.3130331518, -0.0045245460, 0.3957881294, 0.7689774499, 0.9769029553, 0.9355178444, 0.6588835500, 0.2423551776, -0.1965166413, -0.5667722987, -0.8229950948, -0.9581099585, -0.9920534568, -0.9568088180, -0.8860876735, -0.8065847668, -0.7338759424, -0.6742618868, -0.6272951565, -0.5902448424, -0.5600324016, -0.5352180141, -0.5151163517, -0.5000147581, -0.4906514833, -0.4875269645, -0.4916427681, -0.5031516692, -0.5223105210, -0.5485766767, -0.5805718560, -0.6158826052, -0.6511524782, -0.6822064841, -0.7043806433, -0.7138666891, -0.7071659118, -0.6830148273, -0.6411671807, -0.5830774085, -0.5113515367, -0.4287206491, -0.3389323744, -0.2447167818, -0.1494917503, -0.0558721459, 0.0333984994, 0.1167983166, 0.1922442022, 0.2586274831, 0.3154049059, 0.3622669273, 0.3991170106, 0.4264068398, 0.4447118760, 0.4548544831, 0.4582324377, 0.4561073863, 0.4501816844, 0.4421121355, 0.4334047015, 0.4252073945, 0.4181291098, 0.4116712085, 0.4043895489, 0.3933418297, 0.3742298792, 0.3416374979, 0.2897047840, 0.2131285402, 0.1091145371, -0.0228610501, -0.1798448179, -0.3561001095, -0.5439634306, -0.7358882539, -0.9228919112, -1.0955531789, -1.2408738792, -1.3411344033, -1.3732611762, -1.3118793365, -1.1367139370, -0.8423003965, -0.4475547937, 0.0016094210, 0.4393941282, 0.7951118513, 1.0132402636, 1.0660729252, 0.9572622219, 0.7150519086, 0.3798402076, -0.0067217841, -0.4091236075, -0.7978925548, -1.1480862681, -1.4353445860, -1.6353576115, -1.7263161845, -1.6977851977, -1.5591450343, -1.3441773274, -1.1032658082, -0.8861030090, -0.7247741584, -0.6290700686, -0.5947600069, -0.6153700632, -0.6858461167, -0.7961855869, -0.9243107990, -1.0371959589, -1.1038887058, -1.1127499364, -1.0778589638, -1.0307587331, -1.0018330748, -1.0039213584, -1.0260363640, -1.0406849081, -1.0167105999, -0.9315127693, -0.7788119436, -0.5676817354, -0.3173762168, -0.0520960993, 0.2014480279, 0.4150436646, 0.5606172988, 0.6161928712, 0.5766970941, 0.4595106529, 0.3035943044, 0.1554180997, 0.0521148531, 0.0061593771, -0.0016020909, -0.0188038457, -0.1073925600, -0.3189572315, -0.6674234682, -1.1130478368, -1.5705094739, -1.9402907154, -2.1468348286, -2.1640449391, -2.0229352453, -1.8090904935, -1.6505989908, -1.6917042602, -2.0368261287, -2.6673783403, -3.3771744534, -3.8044474459, -3.6196311334, -2.7862879356, -1.6491316158, -0.6940555997, -0.1921441444, -0.0656459033, -0.0366946035, 0.1098673053, 0.3436186605, 0.4382820678, 0.2264736674, -0.2035460861, -0.5927443038, -0.7509588735, -0.6751237421, -0.4791999183, -0.2780242869, -0.1343443326, -0.0699381217, -0.0877314470, -0.1783264940, -0.3170867611, -0.4675856318, -0.5952891793, -0.6815007855, -0.7242045287, -0.7322065571, -0.7168604963, -0.6872542092, -0.6493918548, -0.6079604155, -0.5683353485, -0.5367572218, -0.5208493443, -0.5268414581, -0.5578259855, -0.6116194300, -0.6800219388, -0.7499354269, -0.8071443116, -0.8410322987, -0.8465813185, -0.8240304357, -0.7770106178, -0.7114650327, -0.6337406481, -0.5506025372, -0.4680943615, -0.3914051271, -0.3242220246, -0.2692207110, -0.2276202676, -0.1992264870, -0.1836403076, -0.1802811908, -0.1884331925, -0.2084378834, -0.2403613781, -0.2829789091, -0.3334393695, -0.3868722663, -0.4373721965, -0.4792744663, -0.5078629717, -0.5192770118, -0.5095524803, -0.4743909535, -0.4092325792, -0.3122978524, -0.1885119982, -0.0536657514, 0.0671133085, 0.1448808242, 0.1571358172, 0.0955534464, -0.0319927510, -0.2077779471, -0.4120274241, -0.6299611724, -0.8519387025, -1.0680872188, -1.2589716368, -1.3889114572, -1.4078632153, -1.2679271562, -0.9506436366, -0.4918153325, 0.0156677348, 0.4502416841, 0.7102198231, 0.7509127121, 0.5917088986, 0.2923515607, -0.0795355300, -0.4729944662, -0.8593586109, -1.2246591449, -1.5521769286, -1.8076559004, -1.9369080093, -1.8831994955, -1.6227224707, -1.1954440070, -0.7034783465, -0.2697040629, 0.0165011433, 0.1262831741, 0.0805387999, -0.0768938442, -0.2980514094, -0.5360039443, -0.7476453005, -0.9004957757, -0.9815466291, -0.9967970245, -0.9610149086, -0.8876259313, -0.7933041167, -0.7057842437, -0.6546227865, -0.6462264205, -0.6541534553, -0.6405124227, -0.5826668134, -0.4845251130, -0.3730947963, -0.2936343923, -0.3018007654, -0.4489910125, -0.7623795508, -1.2271100956, -1.7768411142, -2.3001271710, -2.6763603462, -2.8319030547, -2.7768351959, -2.5908669750, -2.3716476255, -2.1815943932, -2.0197302837, -1.8259254044, -1.5254667732, -1.1032381795, -0.6488142773, -0.3040249616, -0.1436900746, -0.1045075879, -0.0260967151, 0.2302399338, 0.6530105409, 1.0315831687, 1.0662681088, 0.6109396057, -0.1729497493, -0.9669617446, -1.5737927522, -1.9502030756, -2.0845770168, -1.9855083725, -1.7282783959, -1.4257192862, -1.1480496183, -0.8802019366, -0.5534232682, -0.1251191082, 0.3472345694, 0.6937958530, 0.7228865026, 0.3736182024, -0.2100344648, -0.7805029634, -1.1619969404, -1.3198668776, -1.3060858589, -1.1822777666, -0.9896434605, -0.7566540722, -0.5120688403, -0.2877418500, -0.1130378484, -0.0107474980, 0.0019844065, -0.0858537068, -0.2745108139, -0.5422087692, -0.8421293411, -1.1127476565, -1.3000128716, -1.3744662611, -1.3346020155, -1.1985868914, -0.9921052112, -0.7398873164, -0.4639959675, -0.1870717101, 0.0620925153, 0.2464283432, 0.3272949951, 0.2781964871, 0.1020138162, -0.1626520997, -0.4512283116, -0.6965437745, -0.8544266928, -0.9145455532, -0.8942999493, -0.8222499559, -0.7240337714, -0.6161511916, -0.5076769440, -0.4049958163, -0.3158907527, -0.2493678555, -0.2115395553, -0.2011713932, -0.2094239570, -0.2256803054, -0.2452619474, -0.2720912118, -0.3146360242, -0.3778075750, -0.4589097015, -0.5499622698, -0.6422931415, -0.7297309391, -0.8084707872, -0.8745765847, -0.9205651577, -0.9339680786, -0.8983204851, -0.7980595518, -0.6269942574, -0.3985571626, -0.1506856258, 0.0606855633, 0.1818458082, 0.1854254413, 0.0803308087, -0.0990371838, -0.3152252422, -0.5449387825, -0.7813831814, -1.0216178710, -1.2486139795, -1.4171822470, -1.4594335559, -1.3124306183, -0.9603529898, -0.4634161100, 0.0494381127, 0.4298487190, 0.5785544448, 0.4877328097, 0.2278803240, -0.1044837807, -0.4357768692, -0.7363810463, -1.0120540085, -1.2773132392, -1.5249655845, -1.7073882410, -1.7470910223, -1.5793375939, -1.2002731695, -0.6839579380, -0.1623317838, 0.2181411731, 0.3566062008, 0.2481855168, -0.0213896601, -0.3360309808, -0.6150399166, -0.8451372019, -1.0708779377, -1.3501455912, -1.6867749582, -1.9787246270, -2.0465734776, -1.7681443622, -1.2055874438, -0.5574028931, -0.0083676399, 0.3129178587, 0.3270982776, 0.0663142431, -0.3113458323, -0.6231241535, -0.7751458037, -0.7983007107, -0.8083644072, -0.9206119298, -1.1531745637, -1.3763036412, -1.3864291542, -1.1029057108, -0.7119856952, -0.5450036749, -0.7596139774, -1.2002726416, -1.6095828790, -1.8850566946, -2.0969726852, -2.3263660662, -2.5185189815, -2.4840272861, -2.0568598327, -1.2791287548, -0.4215802184, 0.2082614416, 0.4745493870, 0.4336743078, 0.2264729995, 0.0025319650, -0.1080026084, -0.0273378822, 0.2359619438, 0.5732841533, 0.7748613638, 0.5072055794, -0.7517224312, -3.7586478844, -9.2850731807, -17.0681320521]
# V1z = [0.9066555421, 1.1855254899, 1.3334417183, 1.8314178830, 2.7717390025, 3.9551997207, 5.0608041730, 5.8620755643, 6.3665130857, 6.7746961490, 7.3111892207, 8.0774036261, 8.9955560885, 9.8361239871, 10.3252321393, 10.2872150307, 9.7126818035, 8.7004904863, 7.3642535494, 5.7906766602, 4.0437217010, 2.1662200436, 0.1831832300, -1.8678920442, -3.8955469863, -5.7347138839, -7.1804689536, -8.0637992566, -8.3274327405, -8.0524734955, -7.4221740452, -6.6499683557, -5.9145908910, -5.3285032893, -4.9366455054, -4.7314907099, -4.6739167252, -4.7117022008, -4.7916636903, -4.8686491689, -4.9098133963, -4.8959809326, -4.8218766798, -4.6939754550, -4.5281154606, -4.3452169325, -4.1670216862, -4.0132318839, -3.8967947865, -3.8242924836, -3.7933124602, -3.7945044920, -3.8125357469, -3.8282157068, -3.8206431817, -3.7700696351, -3.6605460952, -3.4811396828, -3.2276537515, -2.9021716639, -2.5124580535, -2.0701722161, -1.5883226203, -1.0811515714, -0.5609292093, -0.0389437272, 0.4754946189, 0.9741563808, 1.4494003358, 1.8935116260, 2.2974253651, 2.6519467348, 2.9461262847, 3.1687022707, 3.3090632507, 3.3582704123, 3.3099409624, 3.1622448071, 2.9176688336, 2.5849182065, 2.1762356144, 1.7086607293, 1.2016642705, 0.6769123045, 0.1564868381, -0.3363791830, -0.7795443941, -1.1508905580, -1.4307635423, -1.6019988740, -1.6514753325, -1.5712695160, -1.3585885041, -1.0157677381, -0.5494724864, 0.0299988451, 0.7099195103, 1.4761375757, 2.3125812586, 3.2007829523, 4.1193378131, 5.0421182518, 5.9366654202, 6.7635080887, 7.4763237238, 8.0261582575, 8.3675800800, 8.4676565033, 8.3127296352, 7.9109125389, 7.2885696439, 6.4831102714, 5.5342422721, 4.4791012648, 3.3495462708, 2.1720588513, 0.9686346146, -0.2430870853, -1.4485219246, -2.6344327189, -3.7829390787, -4.8634241208, -5.8261495437, -6.6032801927, -7.1225489598, -7.3335452118, -7.2332840191, -6.8800057530, -6.3812363517, -5.8642427320, -5.4389685119, -5.1743211579, -5.0926403328, -5.1784692688, -5.3899802154, -5.6658224290, -5.9325399292, -6.1166626522, -6.1613540989, -6.0392068823, -5.7564080471, -5.3468135934, -4.8622288873, -4.3608466104, -3.8982727408, -3.5215318085, -3.2627775185, -3.1354782534, -3.1294865307, -3.2123472643, -3.3345853090, -3.4432181895, -3.4940309387, -3.4630068916, -3.3515589396, -3.1864632001, -3.0116775466, -2.8765428594, -2.8177094715, -2.8414866653, -2.9136092112, -2.9624895528, -2.9004478430, -2.6592832363, -2.2216478724, -1.6341214485, -0.9862758541, -0.3655140785, 0.1899386839, 0.7164691843, 1.3184362436, 2.1294705113, 3.2562136258, 4.7157769104, 6.3881160890, 7.9961850004, 9.1370058410, 9.3934712856, 8.5172428531, 6.5542172652, 3.8035662170, 0.6674636267, -2.4833288774, -5.3742912461, -7.8260709422, -9.7140618639, -10.9339661761, -11.4222156676, -11.2280692889, -10.5489551230, -9.6725985403, -8.8616544233, -8.2806130999, -7.9901889183, -7.9789978714, -8.1941934462, -8.5599953582, -8.9860144757, -9.3736161610, -9.6256815203, -9.6633763541, -9.4472924723, -8.9925616348, -8.3689152800, -7.6825127640, -7.0504460372, -6.5746716732, -6.3271099521, -6.3399063234, -6.6019703362, -7.0575129653, -7.6103501560, -8.1355511481, -8.5034630079, -8.6114151479, -8.4133999236, -7.9314810600, -7.2458711883, -6.4659366068, -5.7014443798, -5.0410381967, -4.5440903788, -4.2403984984, -4.1354182109, -4.2150160348, -4.4502951101, -4.8006038224, -5.2148727617, -5.6353180778, -6.0003319743, -6.2510516679, -6.3381323047, -6.2286806821, -5.9102868134, -5.3900163067, -4.6906311594, -3.8421199476, -2.8759595657, -1.8207659464, -0.7011616252, 0.4593433164, 1.6339948524, 2.7866551952, 3.8678651989, 4.8149225748, 5.5574422966, 6.0297154743, 6.1869490603, 6.0168062607, 5.5426249036, 4.8136950126, 3.8902126062, 2.8264334070, 1.6614441031, 0.4160418219, -0.9022070644, -2.2906936877, -3.7399284311, -5.2210932525, -6.6747353233, -8.0045451936, -9.0863196622, -9.7934192397, -10.0370049490, -9.7985790338, -9.1367393501, -8.1619350103, -6.9953220877, -5.7350188351, -4.4421682951, -3.1419081065, -1.8346192630, -0.5053936896, 0.8663793671, 2.2963346555, 3.7806428084, 5.2739868976, 6.6696032809, 7.7975055333, 8.4621405176, 8.5183528657, 7.9462553084, 6.8716133778, 5.5182266784, 4.1303108725, 2.9151626979, 2.0229208938, 1.5520794207, 1.5600665492, 2.0677896898, 3.0478483453, 4.3997043232, 5.9221100957, 7.3145651563, 8.2394189837, 8.4474573193, 7.9008332973, 6.7955456321, 5.4621060731, 4.2260739655, 3.3168059580, 2.8473121075, 2.8326335080, 3.2132673109, 3.8640753900, 4.5976863658, 5.1842138936, 5.4072089235, 5.1450390272, 4.4237718249, 3.3883132743, 2.2049173178, 0.9702591579, -0.3092286931, -1.6599786731, -3.0488942745, -4.3231580135, -5.2319454965, -5.5383591959, -5.1591290266, -4.2179721212, -2.9685382887, -1.6561123123, -0.4143513358, 0.7762465481, 2.0829812221, 3.7633313750, 6.0260063439, 8.8159033169, 11.6289670457, 13.5575017328, 13.7554757188, 12.1002801977, 9.3335257772, 6.4762087136, 4.2070693231, 2.6915291331, 1.7292169188, 0.9725271457, 0.0972058336, -1.1085208467, -2.7191925392, -4.6704773515, -6.7497936009, -8.5956311015, -9.7745435027, -9.9636236637, -9.1334156011, -7.5663035476, -5.6966420940, -3.9212685707, -2.5045199899, -1.5787845506, -1.1856535320, -1.3153121357, -1.9264224957, -2.9460360481, -4.2564526704, -5.6810837079, -6.9858799030, -7.9149980689, -8.2657036078, -7.9661530275, -7.1019818361, -5.8672258045, -4.4766639735, -3.0937816566, -1.8012718615, -0.6088225159, 0.5259115791, 1.6728448559, 2.9063375005, 4.2860614797, 5.8295074990, 7.4837610088, 9.1000428271, 10.4375273444, 11.2202692081, 11.2467230670, 10.4909075884, 9.1176011647, 7.3968948326, 5.5882955933, 3.8663029549, 2.3045874087, 0.8967813793, -0.4113813813, -1.6893812800, -2.9928940682, -4.3357709290, -5.6675063614, -6.8653472784, -7.7550833935, -8.1656116946, -8.0006970482, -7.2852287196, -6.1541693869, -4.7928812536, -3.3687606674, -1.9882911892, -0.6876358099, 0.5544802891, 1.7946151586, 3.0983198207, 4.5127565281, 6.0382185972, 7.6018443710, 9.0460518326, 10.1512148334, 10.7034816767, 10.5863257211, 9.8379055874, 8.6304920506, 7.1869156621, 5.6940911042, 4.2563250168, 2.8932493296, 1.5613134605, 0.1829195566, -1.3250929378, -3.0208882559, -4.8997273038, -6.8505341431, -8.6324431305, -9.9104859805, -10.3802463385, -9.9259187639, -8.6897703413, -6.9908254008, -5.1659355711, -3.4512999218, -1.9497644338, -0.6535368349, 0.5161055784, 1.6760857373, 2.9539386829, 4.4575645848, 6.2282553901, 8.1764531711, 10.0300678060, 11.3656498982, 11.7796910821, 11.1247976776, 9.6126243493, 7.6816823580, 5.7644299709, 4.1335882248, 2.8773494825, 1.9456982044, 1.2041495626, 0.4623915544, -0.5182566329, -2.0090760730, -4.2378993311, -7.2083762974, -10.4526660101, -12.9557000262, -13.6162155766, -12.1028783488, -9.1635048121, -5.9838203008, -3.4192164401, -1.7568814764, -0.8666000254, -0.4259178006, -0.0720639983, 0.5349988938, 1.6937423319, 3.6222165740, 6.3366891573, 9.4484285699, 12.0545236289, 13.0635660177, 11.9717306061, 9.3185768623, 6.2151041635, 3.5529011737, 1.6838023283, 0.5260140690, -0.2524342830, -1.0878947040, -2.3676188192, -4.2589664834, -6.5875911905, -8.8450588262, -10.3746576143, -10.7077132857, -9.8380783766, -8.1828666836, -6.2860715824, -4.5591608556, -3.2102071009, -2.3034581040, -1.8539556499, -1.9098178696, -2.5970112980, -4.0816346151, -6.3943421952, -9.1208017156, -11.2269233317]


# ─────────────────────────────────────
# 4. GRAPE optimization
# ─────────────────────────────────────
history=[]
for ep in range(n_epochs+1):
    lr = 1e-6 if ep<=500 else 1e-6

    if ep>0:
        f0_base,_ = f0_and_trace(V1x,V1y,V1z,
                                 theta_rot,tau,N_slices,
                                 B0,omega0,h_FD,J,projs)
        grads=[]
        eps=1e-5
        for arr in (V1x,V1y,V1z):
            g=np.zeros_like(arr)
            for i in range(N_slices):
                arr[i]+=eps
                f_pert,_=f0_and_trace(V1x,V1y,V1z,
                                      theta_rot,tau,N_slices,
                                      B0,omega0,h_FD,J,projs)
                g[i]=(f_pert-f0_base)/eps
                arr[i]-=eps
            grads.append(g)
        V1x+=lr*grads[0]; V1y+=lr*grads[1]; V1z+=lr*grads[2]

    f0_val,tr_inv=f0_and_trace(V1x,V1y,V1z,
                               theta_rot,tau,N_slices,
                               B0,omega0,h_FD,J,projs)
    history.append(tr_inv)
    if ep%10==0:
        print(f"Ep {ep:04d}: f0={f0_val:.6e}  Tr[Fcl^-1]={tr_inv:.10e}")

# ─────────────────────────────────────
# 5. Results
# ─────────────────────────────────────
print("\n# Optimised control fields:")
print("V1x =",V1x)
print("V1y =",V1y)
print("V1z =",V1z)

plt.plot(history); plt.xlabel("epoch"); plt.ylabel("Tr[F_cl^-1]")
plt.title("Training progress (B, ω)"); plt.grid(); plt.show()
